In [ ]:
!pip install geopy

In [ ]:
import pandas as pd

In [ ]:
dir_raw_id = 41
dir_process_id = 42

In [ ]:
from dsmlibrary.datanode import DataNode

datanode = DataNode()

In [ ]:
meta, fp = datanode.get_file(datanode.get_file_id(directory_id=dir_raw_id, name='Electric_Vehicle_Detail.csv'))
meta

In [ ]:
data = pd.read_csv(fp,encoding="utf-8-sig")
data

In [ ]:
data.isnull().sum()

In [ ]:
data2 = data.fillna(0)

data2

In [ ]:
# create DC from CHAdeMo and CCS
data2['DC'] = data2['CHAdeMO'] + data2['CSS2']
# drop CHAdeMo and CCS
data2 = data2.drop(['CHAdeMO','CSS2'], axis=1)
# Charger from AC and DC
data2['Charger'] = data2['AC'] + data2['DC']
data2['Longitude'] = data2['Longitude'].apply(lambda x: x.replace(',',''))
data2

In [ ]:
from geopy.geocoders import Nominatim
from geopy.point import Point

In [ ]:
geolocator = Nominatim(user_agent="geoapiExercises")
def reverse_geocoding(row):
    coord = f"{row['Lattitude']}, {row['Longitude']}"
    location = geolocator.reverse(coord, exactly_one=True, language = 'en')
    address = location.raw['address']
    return address.get('state', '')

In [ ]:
data2['Province'] = data2.apply(reverse_geocoding, axis=1)
data2

In [ ]:
df2 = data2.copy()
df2['Province'] = df2['Province'].apply(lambda x: x.replace('Province',''))
df2

# Write DataNode to DataPlatform


In [ ]:
datanode.write(df=df2, directory=dir_process_id, name="Electric_Vehicle_Detail",
               profiling=True, lineage=[datanode.get_file_id(directory_id=dir_raw_id, name='Electric_Vehicle_Detail.csv')])